<a href="https://colab.research.google.com/github/blancavazquez/CursoDatosMasivosI/blob/master/notebooks/4b_Filtro_Bloom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Filtro de Bloom

En esta libreta programaremos un filtro de Bloom usando NumPy. Un filtro de Bloom consiste en un arreglo de \\(m\\) bits inicializados con  \\(0\\).

* Construcción
  1. Para cada elemento \\(s\\) del conjunto, se calculan los valores _hash_ con \\(k\\) funciones distintas \\(h_1(s), h_2(s), \ldots, h_k(s)\\).
  2. Los \\(k\\) bits en las posiciones correspondientes a los \\(k\\) valores _hash_ se ponen a 1.
  
* Verificación de pertenencia de un nuevo elemento \\(\tilde{s}\\)
  1. Calcula los valores _hash_ para \\(\tilde{s}\\): \\(h_1(\tilde{s}), h_2(\tilde{s}), \ldots , h_k(\tilde{s})\\).
  2. Si todos los bits en las posiciones correspondientes a los \\(k\\) valores _hash_ son 1, entonces el elemento \\(\tilde{s}\\) sí pertenece al conjunto, en caso contrario no pertenece.
  
  
Esta libreta está basada del material del Dr. Gibran Fuentes

In [1]:
!pip install murmurhash

In [2]:
import numpy as np
import murmurhash
import warnings
warnings.filterwarnings('ignore')

class FiltroBloom:  
  def __init__(self, n, m, k):  
    self.n = n
    self.m = m
    self.k = k
    self.arreglo = np.zeros(n, dtype=np.bool)

  def construir(self, s):
    for i in range(self.k):
      index = murmurhash.mrmr.hash(s, i) % self.n #tener índices dentro del arreglo
      self.arreglo[index] = True 

  def verifica(self, s):
    bits = np.zeros(self.k, dtype=np.bool)
    #print(bits)
    for i in range(self.k):
      index = murmurhash.mrmr.hash(s,i) % self.n 
      bits[i] = self.arreglo[index]
      #print(bits)
    return np.all(bits) #si todos los elementos son "1" regresa True, si alguno de los elementos es "0" regresa "False"

## Descarga de URLS populares

In [3]:
!wget https://gist.githubusercontent.com/demersdesigns/4442cd84c1cc6c5ccda9b19eac1ba52b/raw/cf06109a805b661dd12133f9aa4473435e478569/craft-popular-urls

--2022-04-20 17:12:01--  https://gist.githubusercontent.com/demersdesigns/4442cd84c1cc6c5ccda9b19eac1ba52b/raw/cf06109a805b661dd12133f9aa4473435e478569/craft-popular-urls
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2254 (2.2K) [text/plain]
Saving to: ‘craft-popular-urls.13’

craft-popular-urls. 100%[===================>]   2.20K  --.-KB/s    in 0s      

2022-04-20 17:12:01 (25.3 MB/s) - ‘craft-popular-urls.13’ saved [2254/2254]



Leemos la lista de URLs

In [4]:
with open('craft-popular-urls') as f:
  urls = f.read().split('\n')
print(urls)

['http://www.youtube.com', 'http://www.facebook.com', 'http://www.baidu.com', 'http://www.yahoo.com', 'http://www.amazon.com', 'http://www.wikipedia.org', 'http://www.qq.com', 'http://www.google.co.in', 'http://www.twitter.com', 'http://www.live.com', 'http://www.taobao.com', 'http://www.bing.com', 'http://www.instagram.com', 'http://www.weibo.com', 'http://www.sina.com.cn', 'http://www.linkedin.com', 'http://www.yahoo.co.jp', 'http://www.msn.com', 'http://www.vk.com', 'http://www.google.de', 'http://www.yandex.ru', 'http://www.hao123.com', 'http://www.google.co.uk', 'http://www.reddit.com', 'http://www.ebay.com', 'http://www.google.fr', 'http://www.t.co', 'http://www.tmall.com', 'http://www.google.com.br', 'http://www.360.cn', 'http://www.sohu.com', 'http://www.amazon.co.jp', 'http://www.pinterest.com', 'http://www.netflix.com', 'http://www.google.it', 'http://www.google.ru', 'http://www.microsoft.com', 'http://www.google.es', 'http://www.wordpress.com', 'http://www.gmw.cn', 'http://w

Instanciamos la clase y registramos las URLs

In [5]:
fb = FiltroBloom(1000, len(urls), 5) #instancia de la clase (m,n,k)

#Recorremos la lista de URLs y las registramos
for u in urls:
  fb.construir(u)

In [6]:
#Revisando el arreglo
#fb.arreglo

In [7]:
print(u'Proporción de bits distintos a 0 = {0}'.format(fb.arreglo.nonzero()[0].size / fb.arreglo.size))

Proporción de bits distintos a 0 = 0.39


In [8]:
print("http://www.youtube.com", fb.verifica('http://www.youtube.com'))
print("http://www.facebook.com", fb.verifica('http://www.facebook.com'))
print("http://www.wikipedia.org", fb.verifica('http://www.wikipedia.org'))

http://www.youtube.com True
http://www.facebook.com True
http://www.wikipedia.org True


In [9]:
print("http://www.unam.mx", fb.verifica('http://www.unam.mx'))
print("http://www.google.es/", fb.verifica('http://www.google.es/'))
print("http://www.google.es", fb.verifica('http://www.google.es'))
print("http://www.twitter.com", fb.verifica('http://www.twitter.com'))
print("https://www.twitter.com", fb.verifica('https://www.twitter.com'))

http://www.unam.mx False
http://www.google.es/ False
http://www.google.es True
http://www.twitter.com True
https://www.twitter.com True
